# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

output_data_file = "../WeatherPy/output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv(output_data_file, encoding = 'utf-8')

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kaitangata,-46.2817,169.8464,49.64,94,85,7.63,NZ,1618316230
1,Staryy Nadym,65.6118,72.6842,32.41,79,65,18.48,RU,1618316230
2,Merritt,50.1123,-120.7942,28.40,86,1,2.30,CA,1618316230
3,Lavrentiya,65.5833,-171.0000,18.82,95,100,11.25,RU,1618316231
4,Hobart,-42.8794,147.3294,55.40,82,90,11.50,AU,1618316231


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

locations = weather_df[["Lat", "Lng"]]
weather_hum = weather_df["Humidity"]

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = weather_hum, dissipating = False, max_intensity = 100, point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
temp_weather_df = weather_df.dropna(how = "any")

temp_weather_df = temp_weather_df.loc[
    (temp_weather_df["Max Temp"] <= 90)
    & (temp_weather_df["Max Temp"] >= 60)
    & (temp_weather_df["Wind Speed"] <= 10)
    & (temp_weather_df["Cloudiness"] <= 30)
]

temp_weather_df.reset_index(drop = True, inplace = True)
temp_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Avarua,-21.2078,-159.7750,71.60,94,20,3.44,CK,1618316232
1,Port Elizabeth,-33.9180,25.5701,71.60,83,20,9.22,ZA,1618316235
2,Mar del Plata,-38.0023,-57.5575,66.00,93,0,4.61,AR,1618316022
3,Katherine,-14.4667,132.2667,71.60,68,16,3.44,AU,1618316245
4,Hilo,19.7297,-155.0900,66.20,72,20,5.75,US,1618315968
...,...,...,...,...,...,...,...,...,...
66,Saryagash,41.4511,69.1701,69.80,32,0,4.61,KZ,1618316438
67,Rio Grande,-32.0350,-52.0986,66.20,82,0,2.30,BR,1618316438
68,Plettenberg Bay,-34.0527,23.3716,73.49,67,26,6.24,ZA,1618316441
69,Russkiy Aktash,55.0384,52.1214,68.00,42,0,2.24,RU,1618316444


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = temp_weather_df
hotel_df["Hotel Name"] = ""

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"rankby" : "distance",
          "type" : "lodging",
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    
    coordinates = f"{row['Lat']}, {row['Lng']}"
    params["location"] = coordinates
    
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print(f"{coordinates}: {results[0]['name']} is the closest hotel")
    
    except (KeyError, IndexError):
        print("Hotel info not found")

-21.2078, -159.775: Central Motel is the closest hotel
-33.918, 25.5701: Ameera's Villa is the closest hotel
-38.0023, -57.5575: Balneario Parador Las Toscas is the closest hotel
-14.4667, 132.2667: Katherine Motel is the closest hotel
19.7297, -155.09: Dolphin Bay Hotel is the closest hotel
-32.3667, -54.1833: Cabañas San Jorge is the closest hotel
-23.1333, 150.7333: Pandanus Villas is the closest hotel
21.3906, -158.1547: Polynesia Lodging & Conference is the closest hotel
-33.9258, 18.4232: Strand western cape is the closest hotel
16.8, -93.2667: Casa 4 Vientos is the closest hotel
-34.6177, -68.3301: Caja de Jubilaciones y Pensiones para Abogados y Procuradores Delegación San Rafael is the closest hotel
18.7333, -103.6833: El rranchito michuacan de ocampos is the closest hotel
15.7432, -96.4661: Hotel Posada San Jose is the closest hotel
-21.2, -175.2: Far East lodge is the closest hotel
22.2855, 114.1577: Four Seasons Hotel Hong Kong is the closest hotel
31.61, 34.7642: מגדלי סמד

In [9]:
hotel_df.dropna(how = "any", inplace = True)
hotel_df.reset_index(drop = True, inplace = True)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Avarua,-21.2078,-159.7750,71.6,94,20,3.44,CK,1618316232,Central Motel
1,Port Elizabeth,-33.9180,25.5701,71.6,83,20,9.22,ZA,1618316235,Ameera's Villa
2,Mar del Plata,-38.0023,-57.5575,66.0,93,0,4.61,AR,1618316022,Balneario Parador Las Toscas
3,Katherine,-14.4667,132.2667,71.6,68,16,3.44,AU,1618316245,Katherine Motel
4,Hilo,19.7297,-155.0900,66.2,72,20,5.75,US,1618315968,Dolphin Bay Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
gmaps.configure(api_key = g_key)

hotel_layer = gmaps.symbol_layer(
    locations
    ,fill_color = 'rgba(25, 100, 100, 0.4)'
    ,stroke_color = 'rgba(25, 100, 100, 0.4)'
    ,scale = 3
    ,info_box_content = hotel_info
)

markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(hotel_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))